## Wood's方程

In [19]:
import math

# Quartz
phi_quartz = 0.8
vp_quartz = 6008.4
rho_quartz = 2.65
mu_quartz = 44

# Feldspar
phi_Feldspar = 0.2
K_Feldspar = 75
rho_Feldspar = 2.63
mu_Feldspar = 26

# Calculate K_quartz
vs_quartz = math.sqrt(mu_quartz / rho_quartz)
K_quartz = rho_quartz * vp_quartz**2 - 4/3 * rho_quartz * vs_quartz

K_quartz = K_quartz / 10**6
print("K_quartz: ", K_quartz, "GPa")

K = []
K.append(K_quartz)
K.append(K_Feldspar)

phi = []
phi.append(phi_quartz)
phi.append(phi_Feldspar)

# Calculate average K
res_K = 0
for ki, phii in zip(K, phi):
    res_K = res_K + phii / ki

res_K = 1 / res_K


print("K: ", res_K, "GPa")

# Calculate average rho 
Rho = []
Rho.append(rho_quartz)
Rho.append(rho_Feldspar)

rho = 0
for phii, rhoi in zip(phi, Rho):
    rho = rho + phii * rhoi

print("rho: ", rho, "g/cm^3")

# Calculate wood's Velocity
Velocity_wood = math.sqrt(res_K / rho)

print("wood's P wave Velocity: ", Velocity_wood, "km/s")

K_quartz:  95.66729258646934 GPa
K:  90.67020547847233 GPa
rho:  2.646 g/cm^3
wood's P wave Velocity:  5.853793256898434 km/s


## Wyllie时间平均方程

In [21]:
phi = 0.3
V_ma = 5.8537
V_fl = 1.49

V_rock = (1 - phi) / V_ma + phi / V_fl
V_rock = 1 / V_rock

print("V_rock: ", V_rock)

V_rock:  3.1159950841517485


In [23]:
phi = 0.3
varphi1 = 0.8
varphi2 = 0.2
rho_water = 1

rho = (1 - phi) * (varphi1 * rho_quartz + varphi2 * rho_Feldspar) + phi * rho_water
print("average rho: ", rho)

# Solve for V_P
vp = (rho / 0.31)**(1/0.25)

print("V_P:", vp)

average rho:  2.1521999999999997
